In [9]:
# %pip install cython
# %pip install sortedcontainers
# %pip install zipp==3.1.0 --upgrade
# %pip install scorer
# %pip install -r re

In [10]:
import os
import sys
from pathlib import Path

# Если запускаемся на DataSphere
if str(Path.home()) == "/home/jupyter":
    os.chdir("/home/jupyter/work/resources/wunder_summer/examples/datasphere/catboost")
    sys.path.append("/home/jupyter/work/resources/wunder_summer/scorer/orderbook_fast")
    sys.path.append("/home/jupyter/work/resources/wunder_summer/scorer")
    sys.path.append("/home/jupyter/work/resources/wunder_summer/examples/catboost_local")
    data_dir = Path("/home/jupyter/work/resources/wunder_summer/data")
print(Path.cwd())

/home/jupyter/work/resources/wunder_summer/examples/datasphere/catboost


In [11]:
# %pip install -r datasphere_requirements.txt --upgrade
# %pip install -r /home/jupyter/work/resources/wunder_summer/requirements.txt --upgrade
# %pip install -r /home/jupyter/work/resources/wunder_summer/scorer_packages.txt --upgrade

In [12]:
import pyximport
# import scorer
import pystan
import numpy as np
import pandas as pd
# import lightgbm as lg
import catboost
import zipp
from catboost.utils import get_gpu_device_count
from tqdm import tqdm
from pathlib import Path

import orderbook_fast as ob
from my_orderbook import MyOrderBook

print("curdir = ", Path.cwd())
print("homedir = ", Path.home())




SIDE_BID = 0
SIDE_ASK = 1

curdir =  /home/jupyter/work/resources/wunder_summer/examples/datasphere/catboost
homedir =  /home/jupyter


## Импортируем датасет для тренировки модели

## Обучаем модель градиентного бустинга

In [14]:
#!L

X_train = pd.read_pickle(data_dir/"X_train80.pickle")
Y_train = pd.read_pickle("/home/jupyter/work/resources/wunder_summer/data/Y_train80.pickle")

X_test = pd.read_pickle("/home/jupyter/work/resources/wunder_summer/data/X_test20.pickle")
Y_test = pd.read_pickle("/home/jupyter/work/resources/wunder_summer/data/Y_test20.pickle")

print("X_train.shape", X_train.shape)
print("Y_test.shape", Y_test.shape)

train_pool = catboost.Pool(X_train, Y_train, cat_features=[0, 1, 2, 3])
test_pool = catboost.Pool(X_test, Y_test, cat_features=[0, 1, 2, 3])

clf = catboost.CatBoostClassifier(
    learning_rate=0.03,
    depth=10,
    l2_leaf_reg=8,

    bootstrap_type="Bayesian",
    # scale_pos_weight=en.scale_pos_weight,
    # scale_pos_weight=1,
    loss_function = "Logloss",
    eval_metric="AUC:hints=skip_train~false",
    # langevin=True,
    custom_metric=["Recall", "Precision", "Accuracy", "F1", "Kappa", "MCC"],
    # use_best_model=True,
    iterations=5000,
    od_type="Iter",
    od_wait=50,
    rsm=1,
    # random_seed=100,
    task_type="GPU",
    boosting_type='Ordered',
    logging_level="Verbose",
    train_dir="grid",

)


clf.fit(
        X=train_pool,
        plot=False,
        # eval_set=test_pool,
        # use_best_model=True
    )

# Сохраним нашу модель
os.chdir("/home/jupyter/work/resources/wunder_summer/examples/datasphere/catboost")
sys.path.append("/home/jupyter/work/resources/wunder_summer/scorer/orderbook_fast")
sys.path.append("/home/jupyter/work/resources/wunder_summer/scorer")
sys.path.append("/home/jupyter/work/resources/wunder_summer/examples/catboost_local")

print(Path.cwd())
clf.save_model("wunder.model", format="cbm")


X_train.shape (187886, 82)
Y_test.shape (47019, 1)
0:	learn: 0.5267834	total: 44.1ms	remaining: 3m 40s
1:	learn: 0.5268440	total: 82.4ms	remaining: 3m 25s
2:	learn: 0.5267118	total: 123ms	remaining: 3m 25s
3:	learn: 0.5268811	total: 163ms	remaining: 3m 23s
4:	learn: 0.5268915	total: 203ms	remaining: 3m 23s
5:	learn: 0.5268867	total: 241ms	remaining: 3m 20s
6:	learn: 0.5268615	total: 284ms	remaining: 3m 22s
7:	learn: 0.5268470	total: 327ms	remaining: 3m 24s
8:	learn: 0.5268470	total: 364ms	remaining: 3m 21s
9:	learn: 0.5268470	total: 403ms	remaining: 3m 21s
10:	learn: 0.5269128	total: 445ms	remaining: 3m 21s
11:	learn: 0.5268885	total: 486ms	remaining: 3m 21s
12:	learn: 0.5269418	total: 527ms	remaining: 3m 22s
13:	learn: 0.5269392	total: 566ms	remaining: 3m 21s
14:	learn: 0.5268170	total: 603ms	remaining: 3m 20s
15:	learn: 0.5268196	total: 646ms	remaining: 3m 21s
16:	learn: 0.5268042	total: 681ms	remaining: 3m 19s
17:	learn: 0.5268149	total: 720ms	remaining: 3m 19s
18:	learn: 0.5268282	

/py-env/platform-env/lib/python3.7/site-packages/ml_platform/user_messages.py:13: UserWarning: 
The following variables cannot be serialized: test_pool, train_pool, Y_train, X_train, Y_test, X_test

Please note that these variables can be lost in the next working session
  warnings.warn(message)


## Тестируем получившийся классификатор

In [ ]:
# import pystan

In [ ]:
# %pip install cmake
# %pip install scorer==2.0.2 --upgrade --use-feature=2020-resolver --force-reinstall
# %pip install pystan


In [15]:
from my_orderbook import MyOrderBook, action_handler

catboost_myob = my_ob.MyOrderBook()

def process_event_and_predict_proba(ev, orderbook):
    catboost_myob.__getattribute__(action_handler[ev.action])

    if not ev.need_prediction:
        return None

    features = catboost_myob.get_features(ev, orderbook, max_index=13)
    proba = clf.predict_proba([features])[0, 1]
    return proba

from scorer import Scorer

scoring = Scorer("../../../data/train_small_C.npz")
# scoring = Scorer("../../../data/very_small_B.npz")
roc_auc, (true_ys, pred_probas) = scoring.score(process_event_and_predict_proba)

ModuleNotFoundError: No module named 'my_orderbook'

In [ ]:
## Сохраним модель, и решение для отправки готово

## Сохраним модель, и решение для отправки готово

In [39]:
# Сохраним нашу модель
# clf.booster_.save_model("wunder.model")
print(Path.cwd())
clf.save_model("wunder.model", format="cbm")

'''Посмотрите код файла solution.py.
Он использует те же функции что и этот ноутбук, но уже готов к отправке на серверю.
Попробуйте создать архив с файлами solution.py и wunder.model и отправить их на проверку.'''

/home/jupyter/work/resources/wunder_summer/examples/datasphere/catboost


'Посмотрите код файла solution.py.\nОн использует те же функции что и этот ноутбук, но уже готов к отправке на серверю.\nПопробуйте создать архив с файлами solution.py и wunder.model и отправить их на проверку.'

In [ ]:
#!S
